In [1]:
# import libaraies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#import data
nvidia_df = pd.read_csv("/content/drive/MyDrive/NVIDIA_STOCK.csv")
# Display the dataframe
nvidia_df.head()

,Price,Adj Close,Close,High,Low,Open,Volume
0,Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,4.929879665374756,4.983749866485596,4.987500190734863,4.862500190734863,4.894499778747559,355616000
3,2018-01-03,5.254334926605225,5.3117499351501465,5.34250020980835,5.09375,5.102499961853027,914704000
4,2018-01-04,5.2820329666137695,5.339749813079834,5.451250076293945,5.317249774932861,5.394000053405762,583268000


In [3]:
nvidia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1699 non-null   object
 1   Adj Close  1698 non-null   object
 2   Close      1698 non-null   object
 3   High       1698 non-null   object
 4   Low        1698 non-null   object
 5   Open       1698 non-null   object
 6   Volume     1698 non-null   object
dtypes: object(7)
memory usage: 93.0+ KB


In [4]:
# Data Cleaning
nvidia_df = nvidia_df.iloc[2:]  # Remove unwanted rows
nvidia_df = nvidia_df.dropna()  # Remove rows with missing values
nvidia_df[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']] = nvidia_df[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']].astype(float)

In [5]:
# display information
nvidia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 2 to 1698
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      1697 non-null   object 
 1   Adj Close  1697 non-null   float64
 2   Close      1697 non-null   float64
 3   High       1697 non-null   float64
 4   Low        1697 non-null   float64
 5   Open       1697 non-null   float64
 6   Volume     1697 non-null   float64
dtypes: float64(6), object(1)
memory usage: 92.9+ KB


In [6]:
# Missing values
nvidia_df.isna().sum()

,0
Price,0
Adj Close,0
Close,0
High,0
Low,0
Open,0
Volume,0


In [7]:
# Duplicated values
nvidia_df.duplicated().sum()

0

In [8]:
# Summary statistics
nvidia_df.describe()

,Adj Close,Close,High,Low,Open,Volume
count,1697.000000,1697.000000,1697.000000,1697.000000,1697.000000,1.697000e+03
mean,24.797593,24.828411,25.295239,24.315422,24.823341,4.716865e+08
std,29.221442,29.216014,29.811740,28.570804,29.240504,2.093254e+08
min,3.151430,3.177000,3.249500,3.111500,3.162250,9.788400e+07
25%,6.113620,6.161750,6.250000,6.064250,6.166250,3.278440e+08
50%,13.985092,14.015750,14.243500,13.750000,14.033750,4.349650e+08
75%,27.088902,27.104000,27.496000,26.613001,27.077999,5.708320e+08
max,135.559067,135.580002,140.759995,132.419998,139.800003,2.511528e+09
